In [3]:
# report future potential sotcks.

# 오늘부터 ~ 가능한 일자까지 예측치 에서 기준에 부합하느것
import pymysql
import datetime
from datetime import date, timedelta
import configparser
import telepot

LIMIT = 0.70
cf = configparser.ConfigParser()
cf.read('config.cfg')
                               
DB_IP = cf.get('db', 'DB_IP')
DB_USER = cf.get('db', 'DB_USER')
DB_PWD = cf.get('db', 'DB_PWD')
DB_SCH = cf.get('db', 'DB_SCH')
VALID_USER = cf.get('telegram', 'VALID_USER')
TOKEN = cf.get('telegram', 'TOKEN')



conn = pymysql.connect(host=DB_IP, user=DB_USER, password=DB_PWD, db=DB_SCH, charset='utf8mb4')
def get_potential_datas():
    cursor = conn.cursor()           
    cursor.execute("select max(analyzeAt), max(evaluate) from data.forecast;")
    result = cursor.fetchone()
    target_at = result[0] - timedelta(days=result[1])
    query = "select type, code, analyzeAt, potential, volume from data.forecast where analyzeAt > %s and potential > %s order by analyzeAt, code asc"    
    cursor.execute(query, (target_at, str(LIMIT)))
    return cursor.fetchall()

def send_msg(msg):
    if len(msg) is 0:
        return
    print(msg)
    bot = telepot.Bot(TOKEN)    
    bot.sendMessage(chat_id=VALID_USER, text= msg)
    


In [4]:
msg = ''
for data in get_potential_datas():
    msg += (data[2].strftime("%Y-%m-%d") + ' [' +str( data[1]) + '] ['+str(data[3]) + '] [' + str(data[4]) + ']\n')
send_msg(msg)

2017-03-10 [A006280] [1.0] [3272]
2017-03-10 [A020150] [1.0] [1489]
2017-03-10 [A034020] [0.88] [3272]
2017-03-10 [A170900] [1.0] [967]
2017-03-11 [A003410] [0.8] [3272]
2017-03-11 [A004020] [0.75] [3272]
2017-03-11 [A018260] [1.0] [627]
2017-03-11 [A028260] [0.71] [549]
2017-03-11 [A034020] [1.0] [3272]
2017-03-11 [A138930] [0.72] [1471]
2017-03-12 [A000660] [0.72] [3272]

